In [1]:
from random import random
from tabulate import tabulate
from time import time
from math import log, e, sqrt, exp, sin, pi

In [2]:
def estimar_esperanza(N, generador):
    """
    Estima la esperanza usando la ley fuerte de los grandes numeros
    Args:
        N:            Cantidad total de valores a promediar
        generador:    Generador de la variable aleatoria de interes
    """
    esperanza = 0
    for _ in range(N):
        esperanza += generador()
    return esperanza / N

def media_muestral(datos):
    """
    Calcula la media muestral de los datos dados
    """
    n = len(datos)
    media = sum(datos) / n
    return media

def var_muestral(datos):
    """
    Calcula la varianza muestral de los datos dados
    """
    n = len(datos)
    media = media_muestral(datos)
    varianza = sum(list(map(lambda x: (x - media)**2, datos)))
    varianza = varianza / (n - 1)
    return varianza

def generar_muestra(generador, n):
    """
    Genera una muestra de tamaño n de una variable aleatoria
    Args:
        generador:    Generador de la variable aleatoria
        n        :    Tamaño de la muestra
    """
    datos = []
    for _ in range(n):
        datos.append(generador())
    return datos
    
    
def media_muestral_d(d, generador, n=100):
    """
    Estima el valor esperado de la variable aleatoria simulada por generador
    usando el estimador insesgado media muestral.
    Args:
        n:       : Tamaño minimo de la muestra
        d        : Cota para la desviacion aceptable del estimador
        generador: Generador de la variable aleatoria 
    Return:
        media    : Estimacion del valor esperado
        scuad    : Varianza muestral obtenida
        i        : Tamaño de la muestra final
    """
    media = generador()
    scuad = 0
    i = 1 
    while i <= n or sqrt(scuad/i) > d:
        i += 1
        X  = generador()
        media_previa = media
        media = media_previa + (X - media_previa) / i
        scuad = scuad * (1 - 1/(i-1)) + i*(media - media_previa) ** 2
    return media, scuad, i

def estimar_proporcion_d(d, generador, n=100):
    """
    Estima la proporción de exito de una bernoulli, con un desvio 
    de estimador acotado
    c = 1.96 si el intervalo de confianza es del 95%,
        2.33 si el intervalo de confianza es del 98%,
        2.58 si el intervalo de confianza es del 99%,
        1.64 si el intervalo de confianza es del 90%.
    Si L amplitud intervalo d = L / 2 * c
    Args:
        n:       : Tamaño minimo de la muestra
        d        : Cota para la desviacion aceptable del estimador
        generador: Generador de una bernoulli 
    Return:
        p        : Estimacion de la proporcion
        var_m    : Varianza muestral obtenida
        i        : Tamaño de la muestra final
    """
    p = 0
    i = 0
    while i <= n or sqrt(p * (1 - p) / i) > d:
        i += 1
        X = generador()
        p = p + (X - p) / i
    var_m = p * (1 - p)
    return p, var_m, i

In [4]:
## Ejercicio 1
def exponencial(lamda):
    U = 1-random()
    return -log(U)/lamda

def normal_AYR(mu=0, sigma=1):
    while True:
        Y1 = exponencial(1)
        Y2 = exponencial(1)
        if ((Y2 - ((Y1 - 1)**2)/2) >= 0):
            U = random()
            if (U <= 1/2):
                Z = Y1 * sigma + mu
            else:
                Z = -Y1 * sigma + mu
            return Z


tamaño_muestra_esperado = 101
n = 100
d = 0.1

simulacion_1 = media_muestral_d(d, normal_AYR, n)
tamaño_muestra_obtenido = simulacion_1[2]
media_muestral_1    = round(simulacion_1[0], 4)
varianza_muestral_1 = round(simulacion_1[1], 4)

datos_1 = [[tamaño_muestra_esperado, tamaño_muestra_obtenido,
            media_muestral_1, varianza_muestral_1]]

headers_1 = ["Tamaño esperado", "Tamaño obtenido", "Media muestral", 
             "Varianza muestral"]

print(tabulate(datos_1, headers=headers_1, tablefmt="pretty"))

+-----------------+-----------------+----------------+-------------------+
| Tamaño esperado | Tamaño obtenido | Media muestral | Varianza muestral |
+-----------------+-----------------+----------------+-------------------+
|       101       |       101       |     0.0572     |      1.0099       |
+-----------------+-----------------+----------------+-------------------+


In [8]:
## Ejercicio 2

def g_1():
  u = random()
  return exp(u) / sqrt(2*u)

def g_2():
  u = random()
  return ((1/u) - 1)**2 * exp(-((1/u) - 1)**2) * 1/(u**2)

def MonteCarlo(fun, Nsim):
    Integral = 0
    for _ in range(Nsim):
        Integral += fun()
    return Integral/Nsim

d = 0.01
estimacion_i  = media_muestral_d(d, g_1)
estimacion_ii = media_muestral_d(d, g_2)

datos_2   = [["i", estimacion_i[0], estimacion_i[2], MonteCarlo(g_1,100)],
             ["ii", estimacion_ii[0]*2, estimacion_ii[2], MonteCarlo(g_2,100)]
            ]
headers_2 = ["Inciso", "Estimacion", "Datos generados", "MonteCarlo"]

print(tabulate(datos_2, headers=headers_2, tablefmt="pretty"))

+--------+--------------------+-----------------+--------------------+
| Inciso |     Estimacion     | Datos generados |     MonteCarlo     |
+--------+--------------------+-----------------+--------------------+
|   i    | 2.071218470435885  |     106267      | 1.9417013353541057 |
|   ii   | 0.8504425038089697 |      3116       | 0.4188793259494173 |
+--------+--------------------+-----------------+--------------------+


In [16]:
## Ejercicio 3
def integral_i():
    U = random()
    return sin((U+1)*pi) / (U+1)

def integral_ii():
    U = random()
    return 1/U**2 * (3 / (3 + (1/U - 1)**4))

# Parametros para estimar las integrales con un intervalo de confianza del 95%
# con el SEMIANCHO no mayor a 0.001 del intervalo
c = 1.96
d = 0.001 / c  

estimacion_i  = media_muestral_d(d, integral_i)
estimacion_ii = media_muestral_d(d, integral_ii)

numero_sim = [1000, 5000, 7000]
sim_3_i    = list(map(lambda n: generar_muestra(integral_i, n), numero_sim))  
sim_3_ii   = list(map(lambda n: generar_muestra(integral_ii, n), numero_sim))

sim_3_i  = list(map(lambda datos: [media_muestral(datos), 
                                   var_muestral(datos), len(datos)],
                    sim_3_i))

sim_3_ii = list(map(lambda datos: [media_muestral(datos),
                                   var_muestral(datos), len(datos)], 
                    sim_3_ii))

sim_3_i.append(estimacion_i)
sim_3_ii.append(estimacion_ii)

intervalo = lambda media, var, n:  [round(media - sqrt(var/n)* c, 4), 
                                    round(media + sqrt(var/n)* c, 4)]

datos_3_i  = list(map(lambda x: [x[2], intervalo(x[0], x[1], x[2]), round(x[0], 4)], sim_3_i))
datos_3_ii = list(map(lambda x: [x[2], intervalo(x[0], x[1], x[2]), round(x[0], 4)], sim_3_ii))
headers_3  = ["N° de simulaciones", "Intervalo", "Valor estimado"]

print("\nInciso i")
print(tabulate(datos_3_i, headers=headers_3, tablefmt="pretty"))
print("\nInciso ii")
print(tabulate(datos_3_ii, headers=headers_3, tablefmt="pretty"))

[[-0.4279975696813057, 0.04456066307692266, 1000], [-0.4365173093960459, 0.04412357228980376, 5000], [-0.4349370876472395, 0.04439058304056507, 7000], (-0.43338379193518806, 0.04441904888508242, 170642)]

Inciso i
+--------------------+--------------------+----------------+
| N° de simulaciones |     Intervalo      | Valor estimado |
+--------------------+--------------------+----------------+
|        1000        | [-0.4411, -0.4149] |     -0.428     |
|        5000        | [-0.4423, -0.4307] |    -0.4365     |
|        7000        |  [-0.4399, -0.43]  |    -0.4349     |
|       170642       | [-0.4344, -0.4324] |    -0.4334     |
+--------------------+--------------------+----------------+

Inciso ii
+--------------------+------------------+----------------+
| N° de simulaciones |    Intervalo     | Valor estimado |
+--------------------+------------------+----------------+
|        1000        | [1.3775, 1.497]  |     1.4372     |
|        5000        | [1.4356, 1.4895] |     1.462

In [20]:
## Ejercicio 4
def N():
    n = 0
    suma = 0
    while suma <= 1:
        n += 1
        suma += random()
    return n

Nsim = 1000
c = 1.96
d = 0.025 / (2 * c)
estimacion_e = media_muestral_d(d, N, Nsim)
intervalo = lambda media, var, n:  [round(media - sqrt(var/n)* c, 4), 
                                    round(media + sqrt(var/n)* c, 4)]


datos_4   = [[e, estimacion_e[0], (3*e-e**2)/Nsim, estimacion_e[1]/Nsim, intervalo(estimacion_e[0], estimacion_e[1], estimacion_e[2])]]
headers_4 = ["Valor real e","Estimacion e","Varianza real", "Varianza estimador", "Intervalos de confianza"]
print(tabulate(datos_4, headers=headers_4, tablefmt="pretty"))

+-------------------+--------------------+-----------------------+-----------------------+-------------------------+
|   Valor real e    |    Estimacion e    |     Varianza real     |  Varianza estimador   | Intervalos de confianza |
+-------------------+--------------------+-----------------------+-----------------------+-------------------------+
| 2.718281828459045 | 2.7115498413039916 | 0.0007657893864464861 | 0.0007560431595154076 |     [2.6991, 2.724]     |
+-------------------+--------------------+-----------------------+-----------------------+-------------------------+


In [30]:
## Ejercicio 5
def M():
    u1 = random()
    u2 = random()
    n = 2
    while u1 <= u2:
        n += 1
        u1 = u2
        u2 = random()
    return n

Nsim = 1000
c = 1.96
d = 0.1 / 2*c
estimacion_e = media_muestral_d(d, M, Nsim)
intervalo = lambda media, var, n:  [round(media - sqrt(var/n)* c, 4), 
                                    round(media + sqrt(var/n)* c, 4)]


datos_4   = [[e, estimacion_e[0], (3*e-e**2)/Nsim, estimacion_e[1]/Nsim, intervalo(estimacion_e[0], estimacion_e[1], estimacion_e[2])]]
headers_4 = ["Valor real e","Estimacion e","Varianza real", "Varianza estimador", "Intervalos de confianza"]
print(tabulate(datos_4, headers=headers_4, tablefmt="pretty"))

+-------------------+--------------------+-----------------------+-----------------------+-------------------------+
|   Valor real e    |    Estimacion e    |     Varianza real     |  Varianza estimador   | Intervalos de confianza |
+-------------------+--------------------+-----------------------+-----------------------+-------------------------+
| 2.718281828459045 | 2.6963036963037004 | 0.0007657893864464861 | 0.0007576763236763265 |    [2.6424, 2.7502]     |
+-------------------+--------------------+-----------------------+-----------------------+-------------------------+


In [31]:
## Ejercicio 6
def cae_en_circulo():
    """
    Genera la variable aleatoria bernoulli que modela el hecho de que un punto
    aleatorio del cuadrado [-1,1]x[-1,1] caiga en el circulo unitario
    """
    X = random()*2 - 1    # U(-1, 1)  
    Y = random()*2 - 1    # U(-1, 1)
    return int(X**2 + Y**2 <= 1)

# Estimamos pi por intervalo de confianza al 95% con una cota de 0.1
L = 0.1
c = 1.96         # Valor de la normal para la confianza al 95%
d = L / (8 * c)  # Por justificacion analitica

simular_pi = estimar_proporcion_d(d, cae_en_circulo) 

estimacion_pi  = simular_pi[0] * 4  # Porque la media muestral de cae_en_circulo estima pi/4
tamaño_muestra = simular_pi[2]

data_6    = [[pi, estimacion_pi, tamaño_muestra]]
headers_6 = ["Valor de pi", "Estimacion de pi", "Tamaño final de la muestra"]

print(tabulate(data_6, headers=headers_6, tablefmt="pretty"))

+-------------------+-------------------+----------------------------+
|    Valor de pi    | Estimacion de pi  | Tamaño final de la muestra |
+-------------------+-------------------+----------------------------+
| 3.141592653589793 | 3.182000000000002 |            4000            |
+-------------------+-------------------+----------------------------+
